Step 1. DATA IMPORT AND MERGE 

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_web_data_1 = pd.read_csv(r"C:\Users\chuan\Desktop\Ironhack\Week 5\week 5&6 Project\data\raw\df_final_web_data_pt_1.txt")
#display(df_web_data_1) 

#df_web_data_1['client_id'].nunique()

In [3]:
df_web_data_2 = pd.read_csv(r"C:\Users\chuan\Desktop\Ironhack\Week 5\week 5&6 Project\data\raw\df_final_web_data_pt_2.txt")
#display(df_web_data_2) 

#df_web_data_2['client_id'].nunique()

In [4]:
# 1. Merge web data 1&2 
df_web_data = pd.concat([df_web_data_1, df_web_data_2], ignore_index=True)

# 2. Merge df_web_data with df_experiment_client on client_id to match variation group 
df_experiment_client = pd.read_csv(r"C:\Users\chuan\Desktop\Ironhack\Week 5\week 5&6 Project\data\raw\df_final_experiment_clients.txt")
df_final_web_data = df_web_data.merge(df_experiment_client[['client_id', 'Variation']], on='client_id', how='left')

# Group by client_id and variation, then count occurrences
variation_counts = df_final_web_data.groupby(['client_id', 'Variation']).size().reset_index(name='count')

# Display the first few rows
df_final_web_data

,client_id,visitor_id,visit_id,process_step,date_time,Variation
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07,Test
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,Test
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22,Test
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13,Test
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04,Test
...,...,...,...,...,...,...
755400,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:46:10,NaN
755401,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:45:29,NaN
755402,9668240,388766751_9038881013,922267647_3096648104_968866,step_1,2017-05-24 18:44:51,NaN
755403,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:44:34,NaN


In [5]:
# Basic checks
# unique id 70609
df_final_web_data['client_id'].nunique()

# data types
#df_final_web_data.dtypes

120157

Step 2. DATA CLEANING AND WRANGLING 

In [6]:
# 2. Data cleaning and wrangling 

# 2.1 Map the consistancy of the name of columns 
#df_final_web_data.columns = df_web_data.columns.str.replace('_', ' ').str.title()

# 2.2 Drop Duplicates 
df_final_web_data.duplicated().any() #False, no duplicated rows

# 2.3 Check missing values 
df_final_web_data.isnull().sum() #variation missing values 20109/434096 --> dublicates id in repeat steps 

client_id            0
visitor_id           0
visit_id             0
process_step         0
date_time            0
Variation       434096
dtype: int64

In [7]:
# 2.4 Step counts visual 
step_counts = df_final_web_data.groupby(['client_id', 'process_step']).size()

step_counts_unstacked = step_counts.unstack(fill_value=0)

step_counts_unstacked

process_step,confirm,start,step_1,step_2,step_3
client_id,,,,,
169,1,1,1,1,1
336,0,2,0,0,0
546,1,1,1,1,1
555,1,1,1,1,1
647,1,1,1,1,1
...,...,...,...,...,...
9999729,1,4,3,2,1
9999768,1,3,3,3,2
9999832,0,1,1,0,0


Step 3. DEFINE METRICS AND HYPOTHESIS TESTING 

METRICS 1. COMPLETION RATE

In [8]:
import pandas as pd

# Sort by client_id and date_time
df_final_web_data = df_final_web_data.sort_values(by=['client_id', 'date_time'])

# Function to calculate the completion rate using unique client IDs
def calculate_completion_rate(df_final_web_data, completion_step='confirm'):
    # Ensure 'date_time' is in datetime format
    df_final_web_data['date_time'] = pd.to_datetime(df_final_web_data['date_time'])
    
    # Identify unique clients who completed the process
    completed_clients = df_final_web_data[df_final_web_data['process_step'] == completion_step]['client_id'].unique()
    
    # Mark clients who completed the process
    df_final_web_data['confirm'] = df_final_web_data['client_id'].isin(completed_clients)
    
    # Drop duplicates to ensure unique client IDs
    df_unique_clients = df_final_web_data.drop_duplicates(subset='client_id', keep='first')
    
    # Group by variation to count total and completed clients
    completion_counts = df_unique_clients.groupby('Variation')['confirm'].agg(['sum', 'count'])
    completion_counts.columns = ['completed_clients', 'total_clients']
    
    # Calculate completion rate
    completion_counts['completion_rate'] = completion_counts['completed_clients'] / completion_counts['total_clients']
    
    return completion_counts

# Calculate the completion rate

completion_rate_df = calculate_completion_rate(df_final_web_data)

display(completion_rate_df)

# total_clients with unique_id = 50500 

,completed_clients,total_clients,completion_rate
Variation,,,
Control,15434,23532,0.655873
Test,18687,26968,0.692932


In [9]:
import scipy
print(scipy.__version__)

1.13.1


In [10]:
# 3.1.2 --> Set the hypothesis for Hypothis 1 --> if completion rate is equal?
# H0: The completion rate for the Test group (new design) = the completion rate for the Control group (old design).
# H1: The completion rate for the Test group (new design) != the completion rate for the Control group (old design).

#BY USING PROPORTION Z-TEST 

import pandas as pd
from statsmodels.stats.proportion import proportions_ztest

# Number of successes (completed clients) in each group
successes = completion_rate_df['completed_clients'].values

# Number of observations (total clients) in each group
nobs = completion_rate_df['total_clients'].values

# Conduct the two-proportion z-test
z_stat, p_value = proportions_ztest(successes, nobs)

# Output the results
print(f"Z-statistic: {z_stat}")
print(f"P-value: {p_value}")

# Check if p-value is less than alpha (e.g., 0.05) for significance
alpha = 0.05
if p_value < alpha:
    print("Reject null hypothesis: The completion rate for the Test group is not equal to the completion rate for the Control group.")
else:
    print("Fail to reject null hypothesis: There is no significant difference in completion rates between the Test and Control groups.")

Z-statistic: -8.8745141890702
P-value: 7.023933247581432e-19
Reject null hypothesis: The completion rate for the Test group is not equal to the completion rate for the Control group.


In [11]:
# 3.1.2 --> Set the hypothesis for Hypothis 1 --> Completion Rate with a Cost-Effectiveness Threshold 5%.
# H0: The completion rate for the Test group (new design) is equal to or less than the completion rate for the Control group (old design) increased by 5%.
# H1: The completion rate for the Test group (new design) is greater than the completion rate for the Control group (old design) increased by 5%.

import pandas as pd
from statsmodels.stats.proportion import proportions_ztest
# Example completion rates (calculated previously)
completion_rate_df = pd.DataFrame({
    'variation': ['Control', 'Test'],
    'completed_clients': [15434, 18687], 
    'total_clients': [23532, 26968]  
})

# Calculate the completion rate for the Control group and increase by 5%
control_completion_rate = completion_rate_df.loc[completion_rate_df['variation'] == 'Control', 'completed_clients'].values[0] / completion_rate_df.loc[completion_rate_df['variation'] == 'Control', 'total_clients'].values[0]
adjusted_control_completion_rate = control_completion_rate + 0.05

# Number of successes (completed clients) in each group
successes = completion_rate_df['completed_clients'].values

# Number of observations (total clients) in each group
nobs = completion_rate_df['total_clients'].values

# Calculate the z-statistic and p-value for the one-sided test
z_stat, p_value = proportions_ztest(successes, nobs, value=adjusted_control_completion_rate, alternative='larger')

# Output the results
print(f"Z-statistic: {z_stat}")
print(f"P-value: {p_value}")

# Check if p-value is less than alpha (e.g., 0.05) for significance
alpha = 0.05
if p_value < alpha:
    print("Reject null hypothesis: The completion rate for the Test group is greater than the completion rate for the Control group increased by 5%.")
else:
    print("Fail to reject null hypothesis: The completion rate for the Test group is not significantly greater than the completion rate for the Control group increased by 5%.")

Z-statistic: -177.90747247928385
P-value: 1.0
Fail to reject null hypothesis: The completion rate for the Test group is not significantly greater than the completion rate for the Control group increased by 5%.


METRICS 2. TIME SPENT ON EACH STEP

In [12]:
# 3.2 TIME SPENT ON EACH STEP
# 3.2.1 Calculate time spent on each step 

# Filter Data for Control and Test Groups
control_data = df_final_web_data[df_final_web_data['Variation'] == 'Control']
test_data = df_final_web_data[df_final_web_data['Variation'] == 'Test']

# Calculate Time Differences Between Consecutive Steps
control_data['time_diff'] = control_data.groupby('client_id')['date_time'].diff()
test_data['time_diff'] = test_data.groupby('client_id')['date_time'].diff()

# Aggregate Time Differences for Each Step
control_average_time_per_step = control_data.groupby('process_step')['time_diff'].mean()
test_average_time_per_step = test_data.groupby('process_step')['time_diff'].mean()

# Print or visualize the results
print("Control Group:")
print(control_average_time_per_step)

print("\nTest Group:")
print(test_average_time_per_step)


# make sure alternative hypothsis --> which kind of !=, >, < 

Control Group:
process_step
confirm   0 days 18:21:17.307379630
start     4 days 05:57:44.544592030
step_1    0 days 00:30:38.781647281
step_2    0 days 00:00:39.342259834
step_3    0 days 00:09:35.220494575
Name: time_diff, dtype: timedelta64[ns]

Test Group:
process_step
confirm   1 days 05:00:32.783103231
start     3 days 02:55:49.873986735
step_1    0 days 00:22:19.804570955
step_2    0 days 00:01:51.212471355
step_3    0 days 00:12:18.920516488
Name: time_diff, dtype: timedelta64[ns]


C:\Users\chuan\AppData\Local\Temp\ipykernel_14532\2822793933.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_data['time_diff'] = control_data.groupby('client_id')['date_time'].diff()
C:\Users\chuan\AppData\Local\Temp\ipykernel_14532\2822793933.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['time_diff'] = test_data.groupby('client_id')['date_time'].diff()


In [13]:
# 3.2.2 Set the hypothesis for Hypothis 2 --> time spent on each step 
# H0: There is no significant difference in time spent on each step between the Test and Control groups.
# H1: There is significant difference in time spent on each step between the Test and Control groups.

from scipy.stats import ttest_ind

# Extract time differences for each step for both groups
control_time_diff = control_data.groupby('process_step')['time_diff'].apply(list)
test_time_diff = test_data.groupby('process_step')['time_diff'].apply(list)

# Perform the t-test for each step
t_test_results = {}
for step in control_time_diff.index.intersection(test_time_diff.index):
    t_stat, p_value = ttest_ind(control_time_diff[step].dropna(), test_time_diff[step].dropna())
    t_test_results[step] = {'t_stat': t_stat, 'p_value': p_value}

# Print the t-test results
for step, results in t_test_results.items():
    print(f"Step: {step}")
    print(f"  T-statistic: {results['t_stat']}")
    print(f"  P-value: {results['p_value']}")


AttributeError: 'list' object has no attribute 'dropna'

In [ ]:
# COMPARE TIME SPENT ON EACH STEP 







METRICS 3. ERROR RATES

In [14]:
# 3.3 ERROR RATES 
# 3.3.1 Calculate error rates
# Assign numeric values to process steps to identify backward movements
step_mapping = {
    'start': 1,
    'step_1': 2,
    'step_2': 3,
    'step_3': 4,
    'confirm': 5
}
df_final_web_data['step_num'] = df_web_data['process_step'].map(step_mapping)

# Calculate the difference in step numbers to identify backward steps
df_final_web_data['step_diff'] = df_final_web_data.groupby(['client_id', 'visit_id'])['step_num'].diff()

# Identify backward steps (where step_diff is negative)
df_final_web_data['backward'] = df_final_web_data['step_diff'] < 0

# Calculate the total steps and backward steps for each group
error_stats = df_final_web_data.groupby('Variation').agg(
    total_steps=('process_step', 'count'),
    backward_steps=('backward', 'sum')
).reset_index()

# Calculate the error rate for each group
error_stats['error_rate'] = error_stats['backward_steps'] / error_stats['total_steps']

display(error_stats)

,Variation,total_steps,backward_steps,error_rate
0,Control,143462,9615,0.067021
1,Test,177847,16247,0.091354


In [15]:
# 3.3.2 Set the hypothesis for Hypothis 3 --> error rates 
# H0: There is no significant difference in error rates between the Test and Control groups.
# H1: There is a significant difference in error rates between the Test and Control groups.

from statsmodels.stats.proportion import proportions_ztest

# Number of backward steps (errors) in each group
backward_steps = error_stats['backward_steps'].values

# Total number of steps in each group
total_steps = error_stats['total_steps'].values

# Perform the two-proportion z-test
stat, p_value = proportions_ztest(backward_steps, total_steps)

# Display the results
print(f"Z-test statistic: {stat}, p-value: {p_value}")

# Check if p-value is less than alpha (e.g., 0.05) for significance
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference in error rates between the Test and Control groups.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference in error rates between the Test and Control groups.")

Z-test statistic: -25.204049796278067, p-value: 3.616424613022473e-140
Reject the null hypothesis: There is a significant difference in error rates between the Test and Control groups.


In [16]:
df_final_web_data.head(20)

,client_id,visitor_id,visit_id,process_step,date_time,Variation,confirm,step_num,step_diff,backward
285515,169,201385055_71273495308,749567106_99161211863_557568,start,2017-04-12 20:19:36,NaN,True,1,NaN,False
285514,169,201385055_71273495308,749567106_99161211863_557568,step_1,2017-04-12 20:19:45,NaN,True,2,1.0,False
285513,169,201385055_71273495308,749567106_99161211863_557568,step_2,2017-04-12 20:20:31,NaN,True,3,1.0,False
285512,169,201385055_71273495308,749567106_99161211863_557568,step_3,2017-04-12 20:22:05,NaN,True,4,1.0,False
285511,169,201385055_71273495308,749567106_99161211863_557568,confirm,2017-04-12 20:23:09,NaN,True,5,1.0,False
628456,336,64757908_3400128256,649044751_80905125055_554468,start,2017-06-01 07:26:55,NaN,False,1,NaN,False
628331,336,64757908_3400128256,649044751_80905125055_554468,start,2017-06-01 07:42:43,NaN,False,1,0.0,False
442857,546,475037402_89828530214,731811517_9330176838_94847,start,2017-06-17 10:03:29,NaN,True,1,NaN,False
442856,546,475037402_89828530214,731811517_9330176838_94847,step_1,2017-06-17 10:03:39,NaN,True,2,1.0,False
442855,546,475037402_89828530214,731811517_9330176838_94847,step_2,2017-06-17 10:03:52,NaN,True,3,1.0,False


In [ ]:
# substract instead of add 

# sort by steps = times minues = stay time 

# if, else = condition --> loop 

# metrics not only completion rate 

In [ ]:
# 3. Filter by process step 
#filtered_df_final_web_data_start = df_final_web_data[df_final_web_data['process_step'] == 'start']
#filtered_df_final_web_data_start
#filtered_df_final_web_data_start['Client Id'].nunique()

In [ ]:
# total completion rate 
# error analysis by steps 

In [ ]:
#filtered_df_web_data_step1 = df_web_data[df_web_data['Process Step'] == 'step_1']

#filtered_df_web_data_step1

#filtered_df_web_data_step1['Client Id'].nunique()

In [ ]:
#filtered_df_web_data_step2 = df_web_data[df_web_data['Process Step'] == 'step_2']

#filtered_df_web_data_step2

#filtered_df_web_data_step2['Client Id'].nunique()

In [ ]:
#filtered_df_web_data_step3 = df_web_data[df_web_data['Process Step'] == 'step_3']

#filtered_df_web_data_step3

#filtered_df_web_data_step3['Client Id'].nunique()

In [ ]:
#filtered_df_web_data_confirm = df_web_data[df_web_data['Process Step'] == 'confirm']

#filtered_df_web_data_confirm

#filtered_df_web_data_confirm['Client Id'].nunique()

In [ ]:
df_final_web_data.to_csv('df_final_web', index=False)
print("df_final_web_data saved successfully to 'Vanguard_web_data_quantitative.csv'")